In [1]:
import pandas as pd
import json
import os
import time

In [2]:
def digest_json(filename):
    with open(filename) as json_data:
        raw_data = json.load(json_data)
    return raw_data

In [3]:
def extract_gameweek_data(data):
    gameweek = int(data["gameweek"])
    season = str(data["compSeason"]["label"])
    return gameweek,season

def extract_kickoff_data(data):
    game_start = str(data["label"])
    return game_start

def extract_teams_data(data):
    homeTeamData = data[0]
    awayTeamData = data[1]
    
    hScore = int(homeTeamData["score"])
    aScore = int(awayTeamData["score"])
    
    hName = str(homeTeamData["team"]["club"]["name"])
    aName = str(awayTeamData["team"]["club"]["name"])
    
    hAbbr = str(homeTeamData["team"]["club"]["abbr"])
    aAbbr = str(awayTeamData["team"]["club"]["abbr"])
    
    return hName, hAbbr, hScore, aScore, aAbbr, aName

def extract_ground_data(data):
    
    return match_outcome, attendance

def extract_officials_data(data):
    ref_id = int(data[0]["id"])
    ref_first_name = data[0]["name"]["first"]
    ref_last_name = data[0]["name"]["last"]
    ref_full_name = ref_first_name + " " + ref_last_name
    return ref_id, ref_full_name

def extract_lineup_data(data):
    home_lineup_data = data[0]
    away_lineup_data = data[1]
    hLineup = []
    aLineup = []
    hSubs = []
    aSubs = []
    
    # Get home lineup
    for i in range(0,len(home_lineup_data["lineup"])):
        player = home_lineup_data["lineup"][i]
        player_dict = {}
        try:
            player_dict["pos"] = player["matchPosition"]
        except:
            player_dict["pos"] = str("")
        try:
            player_dict["shirt"] = int(player["matchShirtNumber"])
        except:
            player_dict["shirt"] = int(0)
        player_dict["captain"] = player["captain"]
        try:
            player_dict["pos_info"] = player["info"]["positionInfo"]
        except:
            player_dict["pos_info"] = str("")
        player_dict["DOB"] = player["birth"]["date"]["label"]
        player_dict["age"] = player["age"]
        first_name = player["name"]["first"]
        last_name = player["name"]["last"]
        player_dict["name"] = first_name + " " + last_name
        player_dict["id"] = int(player["id"])
        hLineup.append(player_dict)
    
    # Get away lineup
    for i in range(0,len(away_lineup_data["lineup"])):
        player = away_lineup_data["lineup"][i]
        player_dict = {}
        try:
            player_dict["pos"] = player["matchPosition"]
        except:
            player_dict["pos"] = str("")
        try:
            player_dict["shirt"] = int(player["matchShirtNumber"])
        except:
            player_dict["shirt"] = int(0)
        player_dict["captain"] = player["captain"]
        try:
            player_dict["pos_info"] = player["info"]["positionInfo"]
        except:
            player_dict["pos_info"] = str("")
        player_dict["DOB"] = player["birth"]["date"]["label"]
        player_dict["age"] = player["age"]
        first_name = player["name"]["first"]
        last_name = player["name"]["last"]
        player_dict["name"] = first_name + " " + last_name
        player_dict["id"] = int(player["id"])
        aLineup.append(player_dict)
        
    # Get home subs
    for i in range(0,len(home_lineup_data["substitutes"])):
        
        player = home_lineup_data["substitutes"][i]
        player_dict = {}
        try:
            player_dict["pos"] = player["matchPosition"]
        except:
            player_dict["pos"] = str("")
        try:
            player_dict["shirt"] = int(player["matchShirtNumber"])
        except:
            player_dict["shirt"] = int(0)
        player_dict["captain"] = player["captain"]
        try:
            player_dict["pos_info"] = player["info"]["positionInfo"]
        except:
            player_dict["pos_info"] = str("")
        player_dict["DOB"] = player["birth"]["date"]["label"]
        player_dict["age"] = player["age"]
        first_name = player["name"]["first"]
        last_name = player["name"]["last"]
        player_dict["name"] = first_name + " " + last_name
        player_dict["id"] = int(player["id"])
        hSubs.append(player_dict)
        
    # Get away subs
    for i in range(0,len(away_lineup_data["substitutes"])):
        player = away_lineup_data["substitutes"][i]
        player_dict = {}
        try:
            player_dict["pos"] = player["matchPosition"]
        except:
            player_dict["pos"] = str("")
        try:
            player_dict["shirt"] = int(player["matchShirtNumber"])
        except:
            player_dict["shirt"] = int(0)
        player_dict["captain"] = player["captain"]
        try:
            player_dict["pos_info"] = player["info"]["positionInfo"]
        except:
            player_dict["pos_info"] = str("")
        player_dict["DOB"] = player["birth"]["date"]["label"]
        player_dict["age"] = player["age"]
        first_name = player["name"]["first"]
        last_name = player["name"]["last"]
        player_dict["name"] = first_name + " " + last_name
        player_dict["id"] = int(player["id"])
        aSubs.append(player_dict)
    
    return hLineup, aLineup, hSubs, aSubs

def extract_event_data(data):
    all_events_data = []
    for i in range(len(data)):
        event_data = {}
        event_data["time"] = str(data[i]["clock"]["label"])
        event_type = data[i]["type"]
        if event_type == "G":
            # So it's a goal event
            event_data["type"] = "Goal"
            event_data["scorer_id"] = int(data[i]["personId"])
            try:
                event_data["assist_id"] = int(data[i]["assistId"])
            except:
                event_data["assist_id"] = int(0)
        elif event_type == "PE":
            # So it's a "phase end" event type (end of half?)
            event_data["type"] = "Half End"
        elif event_type == "PS":
            event_data["type"] = "Half Start"
        elif event_type == "S":
            event_data["type"] = "Sub"
            event_desc = str(data[i]["description"])
            event_data["desc"] = event_desc
            sub_team = str(data[i]["teamId"])
            if sub_team == "0":
                event_data["team"] = "Home"
            elif sub_team == "1":
                event_data["team"] = "Away"
            else:
                event_data["team"] = ""
            try:
                event_data["player_id"] = int(data[i]["personId"])
            except:
                event_data["player_id"] = int(0)
        elif event_type == "B":
            # So it's a "card" event type
            event_desc = str(data[i]["description"])
            if event_desc == "Y":
                event_data["type"] = "Yellow Card"
            elif event_desc == "R":
                event_data["type"] = "Red Card"
            elif event_desc == "YR":
                event_data["type"] = "Red Card (2nd Yellow)"
            else:
                event_data["type"] = event_type
            try:
                event_data["player_id"] = int(data[i]["personId"])
            except:
                event_data["player_id"] = int(0)
        elif event_type == "P":
            # So penalty event type
            event_data["type"] = "Penalty"
            event_data["scorer_id"] = int(data[i]["personId"])
            try:
                event_data["assist_id"] = int(data[i]["assistId"])
            except:
                event_data["assist_id"] = int(0)
        else:
            # print("New Event Type ({}) found: Event data: {}".format(event_type,data[i]))
            event_data["type"] = event_type
            try:
                event_data["player_id"] = int(data[i]["personId"])
            except:
                event_data["player_id"] = int(0)
        all_events_data.append(event_data)
    return all_events_data

In [4]:
def convert_match_to_row(data,key):
    row_data = {}
    row_data["match_id"] = int(key)
    row_data["opta_id"] = str(data["altIds"]["opta"])
    # Extract information from "gameweek" dict
    gameweek_data = data["gameweek"]
    row_data["gameweek"], row_data["season"] = extract_gameweek_data(gameweek_data)
    
    # Extract information from "kickoff" dict
    kickoff_data = data["kickoff"]
    row_data["kickoff"] = extract_kickoff_data(kickoff_data)
    
    # Extract information from "provisionalKickoff" dict
    
    # Extract information from "teams" dict
    teams_data = data["teams"]
    row_data["hName"], row_data["hAbbr"], row_data["hScore"], row_data["aScore"], row_data["aAbbr"], row_data["aName"] = extract_teams_data(teams_data)
    
    # Extract information from "ground" dict
    row_data["stadium"] = data["ground"]["name"]
    
    # Extract information from "outcome" value
    row_data["outcome"] = data["outcome"]
    
    # Extract information from "attendance" value
    try:
        row_data["attendance"] = int(data["attendance"])
    except:
        row_data["attendance"] = 0
    
    # Extract information from "clock" dict
    
    # Extract information from "matchOfficials" dict
    matchOfficials_data = data["matchOfficials"]
    row_data["ref_id"], row_data["ref_name"] = extract_officials_data(matchOfficials_data)
    
    # Extract information from "halfTimeScore" dict
    
    # Extract information from "teamLists" dict
    lineup_data = data["teamLists"]
    row_data["hLineup"], row_data["aLineup"], row_data["hSubs"], row_data["aSubs"] = extract_lineup_data(lineup_data)
    
    # Extract information from "events" dict
    events_data = data["events"]
    row_data["all_events"] = extract_event_data(events_data)
    
    return row_data

In [5]:
def convert_json_to_df(filename):
    raw_data = digest_json(filename)
    keys = raw_data.keys()
#     output_columns = ["match_id","opta_id","gameweek","season","kickoff","hName","hAbbr","hScore","aScore","aAbbr","aName",
#               "stadium","outcome","attendance","ref_id","ref_name","hLineup","aLineup","hSubs","aSubs","all_events"]
    dflist = []
    for key in keys:
        row_data = convert_match_to_row(raw_data.get(key),key)
        dflist.append(row_data)
    df = pd.DataFrame(dflist)
    return df

In [6]:
def read_all_data_into_mega_csv(csv_file_link,data_folder):
    filelist = get_file_list(data_folder)
    print("{} files to read".format(len(filelist)))
    dflist = []
    for file in filelist:
        file_df = convert_json_to_df(file)
        dflist.append(file_df)
        # print("Added file {} to dflist".format(file.split("/")[-1]))
    output_df = pd.concat(dflist)
    output_df.to_csv(csv_file_link,index=False)
    
def get_file_list(path):
    file_list = []

    for root, dirs, files in os.walk(path):
        for file in files:
            file_list.append(os.path.join(root,file))
    
    return file_list

In [7]:
t1 = time.time()
base_path = "C:/Users/joeco/Python/fantasy-football-strategy/"
csv_file_link = base_path + "data-cleaning/data_outputs/all_prem_data_raw.csv"
data_folder = base_path + "data-collection/data/"
read_all_data_into_mega_csv(csv_file_link,data_folder)
t2 = time.time()
print("Took {} seconds = {} mins = {} hours".format(round(t2-t1,2) , round((t2-t1)/60,2) , round((t2-t1)/3600,2) ))

589 files to read
Took 5.0 seconds = 0.08 mins = 0.0 hours
